# Revisions - additional analysis

In [1]:
import os
import rasterio
import fiona
import geopandas as gpd
import pandas as pd
from rasterstats import zonal_stats
import numpy as np

/home/mester/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
def mkdir(dir):
    
    if not os.path.exists(dir):
        os.mkdir(dir)    
        
        
path_run = os.getcwd() + '/'
path_data_processed = path_run + 'data_processed/'
path_flood_product_merged = path_data_processed + 'flood_product_merged/'
path_revisions = path_data_processed + 'revisions/'
path_revisions_creator = mkdir(path_revisions)
path_revisions_floods_bi = path_revisions + 'floods_bi/'
path_floods_bi_creator = mkdir(path_revisions_floods_bi)
path_performance_scores_files = path_revisions + 'performance_scores_files/'
path_performance_scores_files_creator = mkdir(path_performance_scores_files)
path_data = path_run + 'data/'
path_population_processed = path_data_processed + 'population_processed/'

In [48]:
testpath = '/home/mester/Documents/Submissions/Paper_TC_Idai_Attribution/data/GeoClaw/factual/2019063S18038_cf-zos_aviso-fes_max.tif'

coast_f_fact = rasterio.open(testpath).read()
np.unique((coast_f_fact))

np.unique(np.round(coast_f_fact,2))[0:10]


array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09],
      dtype=float32)

In [3]:
flood_level_threshold_list = []
#for i in range(10,500+10,10):
for i in range(380,420+1,1):
    flood_level_threshold_list.append(i)
    

In [4]:
flood_factual_temp_profile = rasterio.open(path_flood_product_merged + 'flood_product_merged_cf_max.tif').profile
 
#for tide_temp in ['max']:
for tide_temp in ['no','max','min','mean']:
    print(tide_temp)
    flood_factual_temp = rasterio.open(path_flood_product_merged + f"flood_product_merged_cf_{tide_temp}.tif").read()
    
    for flood_level_threshold_temp in flood_level_threshold_list:

        print(flood_level_threshold_temp)

        flood_factual_temp_bi = flood_factual_temp.copy()

        flood_factual_temp_bi[flood_factual_temp_bi < flood_level_threshold_temp/100] = 0
        flood_factual_temp_bi[flood_factual_temp_bi > 0] = 1

        with rasterio.open(path_revisions_floods_bi + 'flood_product_merged_factual' + '_' + tide_temp + f"_{flood_level_threshold_temp}_bi.tif", "w", **flood_factual_temp_profile) as dest:
            dest.write(flood_factual_temp_bi) 

        flood_factual_temp_bi_shp = path_revisions_floods_bi + 'flood_product_merged_factual' + '_' + tide_temp + f"_{flood_level_threshold_temp}_bi.shp"   
        mask = None
        from rasterio.features import shapes

        with rasterio.Env():

            with rasterio.open(path_revisions_floods_bi + 'flood_product_merged_factual_' + tide_temp + f"_{flood_level_threshold_temp}_bi.tif") as src:
                image = src.read()

            results = (
            {'properties': {'raster_val': v}, 'geometry': s}
            for i, (s, v)
            in enumerate(
            shapes(image, mask=mask, transform=src.transform)))

            with fiona.open(
            flood_factual_temp_bi_shp, 'w',
            driver='Shapefile',
            crs=src.crs,
            schema={'properties': [('raster_val', 'int')],
            'geometry': 'Polygon'}) as dst:
                dst.writerecords(results)

        flood_factual_temp_dissolved = gpd.read_file(flood_factual_temp_bi_shp)
        flood_factual_temp_dissolved = flood_factual_temp_dissolved[flood_factual_temp_dissolved['raster_val'] == 1]
        flood_factual_temp_dissolved.to_file(flood_factual_temp_bi_shp)

no
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
max
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
min
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
mean
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420


In [12]:
# Affected people

path_pop_tif_cropped = path_population_processed + 'MOZ_GHSL_9as_clipped.tif'

scenario_list = []
tide_list = []
flood_level_threshold_results_list = []
affected_list = []

tide = tide_temp

#for tide_temp in ['max']:
for tide_temp in ['no','max','min','mean']:
    print(tide_temp)

    for flood_level_threshold_temp in flood_level_threshold_list:

        print(flood_level_threshold_temp)

        for filename in sorted(os.listdir(path_revisions_floods_bi)): 

            if filename.split('_')[4] == tide_temp and flood_level_threshold_temp == int(filename.split('_')[5]) and filename.split('.')[1] == 'shp':

              #  flood_level_threshold_temp = filename.split('_')[5]

                filename_cut_cf = filename.split('_')[3]

                print('it is a match!')

                print(filename_cut_cf)

                stat = 0


                flood = gpd.read_file(path_revisions_floods_bi + 'flood_product_merged_factual' + '_' + tide_temp + f"_{flood_level_threshold_temp}_bi.shp")

                stat = zonal_stats(flood,
                    path_pop_tif_cropped, 
                    all_touched = False,
                    stats = ['sum'])                           

                stat = int(pd.DataFrame(stat).sum().iloc[0])

                print(stat)

                scenario_list.append(filename_cut_cf)
                tide_list.append(tide_temp)
                flood_level_threshold_results_list.append(flood_level_threshold_temp)
                affected_list.append(stat)

results = pd.DataFrame()
results.loc[:,'scenario'] = scenario_list
results.loc[:,'tide'] = tide_list
results.loc[:,'threshold'] = flood_level_threshold_results_list
results.loc[:,'affected'] = affected_list
#results.to_csv(f"{path_revisions}results_{tide}_{flood_level_threshold}.csv", index=False)
results.to_csv(path_revisions + 'results_new.csv', index=False)

no
380
it is a match!
factual
527111
381
it is a match!
factual
517987
382
it is a match!
factual
517987
383
it is a match!
factual
517987
384
it is a match!
factual
517987
385
it is a match!
factual
517987
386
it is a match!
factual
517987
387
it is a match!
factual
517987
388
it is a match!
factual
517987
389
it is a match!
factual
517987
390
it is a match!
factual
517987
391
it is a match!
factual
505383
392
it is a match!
factual
505383
393
it is a match!
factual
505383
394
it is a match!
factual
505383
395
it is a match!
factual
505383
396
it is a match!
factual
505383
397
it is a match!
factual
505383
398
it is a match!
factual
505383
399
it is a match!
factual
505383
400
it is a match!
factual
505383
401
it is a match!
factual
480838
402
it is a match!
factual
480838
403
it is a match!
factual
480838
404
it is a match!
factual
480838
405
it is a match!
factual
480838
406
it is a match!
factual
480838
407
it is a match!
factual
480838
408
it is a match!
factual
480838
409
it is a

In [60]:
# del

stat = 0


flood1 = gpd.read_file("/home/mester/Downloads/delete_me/new38/flood_product_merged_cf_max_400_bi.shp")

stat = zonal_stats(flood1,
    path_pop_tif_cropped, 
    all_touched = False,
    stats = ['sum'])                           

stat = int(pd.DataFrame(stat).sum().iloc[0])

print(stat)
print(flood.area.sum())

flood2 = gpd.read_file("/home/mester/Downloads/delete_me/new38/flood_product_merged_cfwind_max_400_bi.shp")

stat = zonal_stats(flood2,
    path_pop_tif_cropped, 
    all_touched = False,
    stats = ['sum'])                           

stat = int(pd.DataFrame(stat).sum().iloc[0])

print(stat)
print(flood.area.sum())

475577
0.6401687499988378


/home/mester/anaconda3/envs/ipykernel_py3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  app.launch_new_instance()


485781
0.6401687499988378


/home/mester/anaconda3/envs/ipykernel_py3/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [66]:
dif = gpd.overlay(flood1,flood2,how='difference')
dif.to_file('del_intersection.shp')

dif = gpd.overlay(flood2,flood1,how='difference')
dif.to_file('del_intersection2.shp')

In [9]:
results = pd.read_csv(path_revisions + 'results.csv')

for tide_temp in ['no','max','min','mean']:
    print(tide_temp)
    results_temp = results[results.tide == tide_temp]
    val_closest = min(results_temp.affected, key=lambda x:abs(x-478_000))
    print(results[results.affected == val_closest].threshold.iloc[0])

FileNotFoundError: [Errno 2] No such file or directory: '/home/mester/Documents/Submissions/Paper_TC_Idai_Attribution/data_processed/revisions/results.csv'

In [13]:
results = pd.read_csv(path_revisions + 'results_new.csv')

for tide_temp in ['no','max','min','mean']:
    print(tide_temp)
    results_temp = results[results.tide == tide_temp]
    val_closest = min(results_temp.affected, key=lambda x:abs(x-478_000))
    print(results[results.affected == val_closest].threshold.iloc[0])
  #  print(results[results.affected == val_closest])#.threshold)#.iloc[0])

no
401
max
401
min
401
mean
401


In [16]:
results#[results.affected == val_closest]

,scenario,tide,threshold,affected
0,factual,no,380,527111
1,factual,no,381,517987
2,factual,no,382,517987
3,factual,no,383,517987
4,factual,no,384,517987
...,...,...,...,...
159,factual,mean,416,464472
160,factual,mean,417,464472
161,factual,mean,418,464472
162,factual,mean,419,464472


In [29]:
results[-20:]

,scenario,tide,threshold,affected
64,factual,mean,391,484609
65,factual,mean,392,484609
66,factual,mean,393,484609
67,factual,mean,394,484609
68,factual,mean,395,484609
69,factual,mean,396,484609
70,factual,mean,397,484609
71,factual,mean,398,484609
72,factual,mean,399,484609
73,factual,mean,400,484609


In [23]:
results[-13:]

,scenario,tide,threshold,affected
187,factual,mean,380,506358
188,factual,mean,390,497213
189,factual,mean,400,484609
190,factual,mean,410,460104
191,factual,mean,420,453406
192,factual,mean,430,434024
193,factual,mean,440,426636
194,factual,mean,450,417340
195,factual,mean,460,406493
196,factual,mean,470,378226


In [17]:
results[-13:]

,scenario,tide,threshold,affected
187,factual,mean,380,506358
188,factual,mean,390,497213
189,factual,mean,400,484609
190,factual,mean,410,460104
191,factual,mean,420,453406
192,factual,mean,430,434024
193,factual,mean,440,426636
194,factual,mean,450,417340
195,factual,mean,460,406493
196,factual,mean,470,378226


# Flood difference factual vs counterfactual

In [5]:
factual_fld = rasterio.open(path_data_processed + 'flood_product_merged/flood_product_merged_cf_max.tif').read()
counterfactual_fld = rasterio.open(path_data_processed + 'flood_product_merged/flood_product_merged_cfwindzos105_max.tif').read()
dif_fld = factual_fld - counterfactual_fld
dif_fld[dif_fld==0]=np.nan

profile = rasterio.open(path_data_processed + 'flood_product_merged/flood_product_merged_cf_max.tif').profile

with rasterio.open(path_run + 'revision/flood_difference.tif', "w", **profile) as dest:
    dest.write(dif_fld) 



# Performance scores

In [62]:
# crop satellite imagery and convert to .shp

import rasterio.mask

beira_bounding = path_data_processed + '/QGIS_processed/Beira_districts_bounding.shp'

#mask = None  

path_raster_layer_resampled = path_data_processed + 'RICorDE_processed/idai_r1_0331_depths_0_resampled_9as_reproj.tif'

with fiona.open(beira_bounding, "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

with rasterio.open(path_raster_layer_resampled) as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta

out_meta.update({"driver": "GTiff",
                  "height": out_image.shape[1],
                  "width": out_image.shape[2],
                  "transform": out_transform})

out_image[out_image==0] = np.nan
out_image[out_image>=0.1] = 1
path_raster_layer_resampled_cropped = path_performance_scores_files + 'RICorDE_cropped.tif'

with rasterio.open(path_raster_layer_resampled_cropped, "w", **out_meta) as dest:
        dest.write(out_image) 

# .tif to .shp
path_raster_layer_resampled_cropped_shape = path_performance_scores_files + 'RICorDE_cropped.shp'

from rasterio.features import shapes

with rasterio.Env():
    with rasterio.open(path_raster_layer_resampled_cropped) as src:
        image = src.read()

        results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for i, (s, v)
        in enumerate(
        shapes(image, mask=mask, transform=src.transform)))

        with fiona.open(
        path_raster_layer_resampled_cropped_shape, 'w',
        driver='Shapefile',
        crs=src.crs,
        schema={'properties': [('raster_val', 'int')],
        'geometry': 'Polygon'}) as dst:
            dst.writerecords(results)         

shape_satellite = gpd.read_file(path_raster_layer_resampled_cropped_shape)
shape_satellite = shape_satellite[shape_satellite['raster_val'] == 1]

if len(shape_satellite) != 0:
    shape_satellite = shape_satellite.dissolve(by='raster_val') #cascaded_union
    shape_satellite.to_file(path_raster_layer_resampled_cropped_shape)

In [63]:
# crop coastal_flooding and convert to .shp

for tide_temp in ['no','min','mean','max']:
    
    print(tide_temp)

    import rasterio.mask

    beira_bounding = path_data_processed + '/QGIS_processed/Beira_districts_bounding.shp'

    #mask = None  

    path_raster_layer_resampled = path_data + f"GeoClaw/factual/2019063S18038_cf-zos_aviso-fes_{tide_temp}.tif"

    with fiona.open(beira_bounding, "r") as shapefile:
        shapes = [feature["geometry"] for feature in shapefile]

    with rasterio.open(path_raster_layer_resampled) as src:
        out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
        out_meta = src.meta

    out_meta.update({"driver": "GTiff",
                      "height": out_image.shape[1],
                      "width": out_image.shape[2],
                      "transform": out_transform})

    out_image[out_image==0] = np.nan
    out_image[out_image>=0.1] = 1
    path_raster_layer_resampled_cropped = path_performance_scores_files + f"GeoClaw_{tide_temp}_cropped.tif"

    with rasterio.open(path_raster_layer_resampled_cropped, "w", **out_meta) as dest:
            dest.write(out_image) 

    # .tif to .shp
    path_raster_layer_resampled_cropped_shape = path_performance_scores_files + f"GeoClaw_{tide_temp}_cropped.shp"

    from rasterio.features import shapes

    with rasterio.Env():
        with rasterio.open(path_raster_layer_resampled_cropped) as src:
            image = src.read()

            results = (
            {'properties': {'raster_val': v}, 'geometry': s}
            for i, (s, v)
            in enumerate(
            shapes(image, mask=mask, transform=src.transform)))

            with fiona.open(
            path_raster_layer_resampled_cropped_shape, 'w',
            driver='Shapefile',
            crs=src.crs,
            schema={'properties': [('raster_val', 'int')],
            'geometry': 'Polygon'}) as dst:
                dst.writerecords(results)         

    shape_GeoClaw = gpd.read_file(path_raster_layer_resampled_cropped_shape)
    shape_GeoClaw = shape_GeoClaw[shape_GeoClaw['raster_val'] == 1]

    if len(shape_GeoClaw) != 0:
        shape_GeoClaw = shape_GeoClaw.dissolve(by='raster_val') #cascaded_union
        shape_GeoClaw.to_file(path_raster_layer_resampled_cropped_shape)


    # Performance scores

    shape_satellite_area = np.array(shape_satellite['geometry'].to_crs({'proj':'cea'}).area)
    shape_GeoClaw_area = np.array(shape_GeoClaw['geometry'].to_crs({'proj':'cea'}).area)

    union = gpd.overlay(shape_GeoClaw, shape_satellite, how='union')
    union['unit'] = 0
    union = union.dissolve(by='unit')
    union_area = np.array(union['geometry'].to_crs({'proj':'cea'}).area)
    union.to_file(path_performance_scores_files + f"GeoClaw_{tide_temp}_cropped_union.shp")

    inter = gpd.overlay(shape_GeoClaw, shape_satellite, how='intersection')
    inter_area = np.array(inter['geometry'].to_crs({'proj':'cea'}).area)
    inter.to_file(path_performance_scores_files + f"GeoClaw_{tide_temp}_cropped_inter.shp")

    Fm_area = shape_GeoClaw_area
    Fo_area = shape_satellite_area
    FmAFo_area = inter_area
    FmUFo_area = union_area



    #######################
    ## Score Calculation ##
    #######################

    CSI = FmAFo_area / FmUFo_area                        
    HR = FmAFo_area / Fo_area                        
    Bias = ((FmAFo_area+Fm_area)/(FmAFo_area+Fo_area))-1

    CSI = round(CSI[0],2)
    HR = round(HR[0],2)
    Bias = round(Bias[0],2)

    print("CSI = ", CSI)
    print("HR = ", HR)
    print("Bias = ", Bias)
    print("\n")

no
CSI =  0.3
HR =  0.64
Bias =  0.47


min
CSI =  0.1
HR =  0.11
Bias =  -0.72


mean
CSI =  0.33
HR =  0.54
Bias =  0.13


max
CSI =  0.3
HR =  0.65
Bias =  0.5




In [51]:
print('hi \n ho')

hi 
 ho
